In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# 시각화 한글폰트 설정
# 맑은 고딕

plt.rc('font', family='Malgun Gothic')
sns.set(font="Malgun Gothic",#"NanumGothicCoding", 
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')

# scipy
import scipy.stats as spst

In [2]:
wbc_p = pd.read_csv('./국대_투수.csv', encoding = 'cp949')
wbc_b = pd.read_csv('./국대_타자.csv', encoding = 'cp949')
kbo_p = pd.read_csv('./kbo_투수2.csv', encoding = 'cp949')
kbo_b = pd.read_csv('./kbo_타자2.csv', encoding = 'cp949')
wbc_won_p = pd.read_csv('./원장희_국대_투수.csv', encoding = 'cp949')
wbc_won_b = pd.read_csv('./원장희_국대_타자.csv', encoding = 'cp949')
wbc_lim_p = pd.read_csv('./임채원_국대_투수.csv')
wbc_lim_b = pd.read_csv('./임채원_국대_타자.csv')

In [3]:
kbo_b.replace('kt','KT', inplace=True)
kbo_b.replace('SK','SSG', inplace=True)
kbo_b.replace('넥센','키움', inplace=True)
kbo_p.replace('kt','KT', inplace=True)
kbo_p.replace('SK','SSG', inplace=True)
kbo_p.replace('넥센','키움', inplace=True)


wbc_b.rename(columns = {'WAR*' : 'BWAR', 'WPA' : 'BWPA'}, inplace = True)
wbc_p.rename(columns = {'WAR' : 'PWAR', 'WPA' : 'PWPA'}, inplace = True)
wbc_lim_b.rename(columns = {'WAR*' : 'BWAR', 'WPA' : 'BWPA'}, inplace = True)
wbc_lim_p.rename(columns = {'투수WAR' : 'PWAR', '투수WPA' : 'PWPA'}, inplace = True)

wbc_won_b.rename(columns = {'타자WAR' : 'BWAR', '타자WPA' : 'BWPA'}, inplace = True)
wbc_won_p.rename(columns = {'투수WAR' : 'PWAR', '투수WPA' : 'PWPA'}, inplace = True)
kbo_b.rename(columns = {'타자WAR' : 'BWAR', '타자WPA' : 'BWPA'}, inplace = True)
kbo_p.rename(columns = {'투수WAR' : 'PWAR', '투수WPA' : 'PWPA'}, inplace = True)

# 타자 세이버 지표 추가

In [4]:
#GPA 지표 추가
kbo_b['GPA'] = (kbo_b['출루']*1.8 + kbo_b['장타'])/4
wbc_b['GPA'] = (wbc_b['출루']*1.8 + wbc_b['장타'])/4
wbc_won_b['GPA'] = (wbc_won_b['출루']*1.8 + wbc_won_b['장타'])/4
wbc_lim_b['GPA'] = (wbc_lim_b['출루']*1.8 + wbc_lim_b['장타'])/4


#XR 지표 추가
kbo_b['XR'] = (kbo_b['안타']-kbo_b['2타']-kbo_b['3타']-kbo_b['홈런'])*0.5 + kbo_b['2타']*0.72 + kbo_b['3타']*1.04 + kbo_b['홈런']*1.44 + (kbo_b['볼넷']-kbo_b['사구'])*0.34 + kbo_b['고4']*0.25 + kbo_b['도루']*0.18 -  kbo_b['도실']*0.32 - (kbo_b['타수']-kbo_b['안타']-kbo_b['삼진'])*0.09 - kbo_b['삼진']*0.098 - kbo_b['병살']*0.37 + kbo_b['희비']*0.37 + kbo_b['희타']*0.04
wbc_b['XR'] = (wbc_b['안타']-wbc_b['2타']-wbc_b['3타']-wbc_b['홈런'])*0.5 + wbc_b['2타']*0.72 + wbc_b['3타']*1.04 + wbc_b['홈런']*1.44 + (wbc_b['볼넷']-wbc_b['사구'])*0.34 + wbc_b['고4']*0.25 + wbc_b['도루']*0.18 -  wbc_b['도실']*0.32 - (wbc_b['타수']-wbc_b['안타']-wbc_b['삼진'])*0.09 - wbc_b['삼진']*0.098 - wbc_b['병살']*0.37 + wbc_b['희비']*0.37 + wbc_b['희타']*0.04
wbc_won_b['XR'] = (wbc_won_b['안타']-wbc_won_b['2타']-wbc_won_b['3타']-wbc_won_b['홈런'])*0.5 + wbc_won_b['2타']*0.72 + wbc_won_b['3타']*1.04 + wbc_won_b['홈런']*1.44 + (wbc_won_b['볼넷']-wbc_won_b['사구'])*0.34 + wbc_won_b['고4']*0.25 + wbc_won_b['도루']*0.18 -  wbc_won_b['도실']*0.32 - (wbc_won_b['타수']-wbc_won_b['안타']-wbc_won_b['삼진'])*0.09 - wbc_won_b['삼진']*0.098 - wbc_won_b['병살']*0.37 + wbc_won_b['희비']*0.37 + wbc_won_b['희타']*0.04
wbc_lim_b['XR'] = (wbc_lim_b['안타']-wbc_lim_b['2타']-wbc_lim_b['3타']-wbc_lim_b['홈런'])*0.5 + wbc_lim_b['2타']*0.72 + wbc_lim_b['3타']*1.04 + wbc_lim_b['홈런']*1.44 + (wbc_lim_b['볼넷']-wbc_lim_b['사구'])*0.34 + wbc_lim_b['고4']*0.25 + wbc_lim_b['도루']*0.18 -  wbc_lim_b['도실']*0.32 - (wbc_lim_b['타수']-wbc_lim_b['안타']-wbc_lim_b['삼진'])*0.09 - wbc_lim_b['삼진']*0.098 - wbc_lim_b['병살']*0.37 + wbc_lim_b['희비']*0.37 + wbc_lim_b['희타']*0.04

#XR27 지표 추가
kbo_b['XR27'] = (kbo_b['XR']/(kbo_b['타석']-kbo_b['안타']+kbo_b['희타']+kbo_b['희비']+kbo_b['도실']+kbo_b['병살'])) * 27
wbc_b['XR27'] = (wbc_b['XR']/(wbc_b['타석']-wbc_b['안타']+wbc_b['희타']+wbc_b['희비']+wbc_b['도실']+wbc_b['병살'])) * 27
wbc_won_b['XR27'] = (wbc_won_b['XR']/(wbc_won_b['타석']-wbc_won_b['안타']+wbc_won_b['희타']+wbc_won_b['희비']+wbc_won_b['도실']+wbc_won_b['병살'])) * 27
wbc_lim_b['XR27'] = (wbc_lim_b['XR']/(wbc_lim_b['타석']-wbc_lim_b['안타']+wbc_lim_b['희타']+wbc_lim_b['희비']+wbc_lim_b['도실']+wbc_lim_b['병살'])) * 27

#RC27 지표 추가
#RC = A*B/C  A = 안타+볼넷+사구-도실-병살, B = 루타 + 0.26(볼넷-고4+사구) + 0.52*(희타+희비+도루), C = 타석 + 볼넷 + 사구 + 희타 + 희비 
#RC27 = RC/(타석-안타+희타+희비+도실+병살)*27

kbo_b['A'] = kbo_b['안타'] + kbo_b['볼넷'] + kbo_b['사구'] - kbo_b['도실'] - kbo_b['병살']
kbo_b['B'] = kbo_b['루타'] + 0.26*(kbo_b['볼넷'] - kbo_b['고4'] + kbo_b['사구']) + 0.52*(kbo_b['희타'] + kbo_b['희비'] + kbo_b['도루'])
kbo_b['C'] = kbo_b['타석'] + kbo_b['볼넷'] + kbo_b['사구'] + kbo_b['희타'] + kbo_b['희비']
kbo_b['RC'] = kbo_b['A']*kbo_b['B']/kbo_b['C']
kbo_b['RC27'] = (kbo_b['RC']/(kbo_b['타석'] - kbo_b['안타'] + kbo_b['희타'] + kbo_b['희비'] + kbo_b['도실'] + kbo_b['병살']))*27



wbc_b['A'] = wbc_b['안타'] + wbc_b['볼넷'] + wbc_b['사구'] - wbc_b['도실'] - wbc_b['병살']
wbc_b['B'] = wbc_b['루타'] + 0.26*(wbc_b['볼넷'] - wbc_b['고4'] + wbc_b['사구']) + 0.52*(wbc_b['희타'] + wbc_b['희비'] + wbc_b['도루'])
wbc_b['C'] = wbc_b['타석'] + wbc_b['볼넷'] + wbc_b['사구'] + wbc_b['희타'] + wbc_b['희비']
wbc_b['RC'] = wbc_b['A']*wbc_b['B']/wbc_b['C']
wbc_b['RC27'] = (wbc_b['RC']/(wbc_b['타석'] - wbc_b['안타'] + wbc_b['희타'] + wbc_b['희비'] + wbc_b['도실'] + wbc_b['병살']))*27


wbc_won_b['A'] = wbc_won_b['안타'] + wbc_won_b['볼넷'] + wbc_won_b['사구'] - wbc_won_b['도실'] - wbc_won_b['병살']
wbc_won_b['B'] = wbc_won_b['루타'] + 0.26*(wbc_won_b['볼넷'] - wbc_won_b['고4'] + wbc_won_b['사구']) + 0.52*(wbc_won_b['희타'] + wbc_won_b['희비'] + wbc_won_b['도루'])
wbc_won_b['C'] = wbc_won_b['타석'] + wbc_won_b['볼넷'] + wbc_won_b['사구'] + wbc_won_b['희타'] + wbc_won_b['희비']
wbc_won_b['RC'] = wbc_won_b['A']*wbc_won_b['B']/wbc_won_b['C']
wbc_won_b['RC27'] = (wbc_won_b['RC']/(wbc_won_b['타석'] - wbc_won_b['안타'] + wbc_won_b['희타'] + wbc_won_b['희비'] + wbc_won_b['도실'] + wbc_won_b['병살']))*27

wbc_lim_b['A'] = wbc_lim_b['안타'] + wbc_lim_b['볼넷'] + wbc_lim_b['사구'] - wbc_lim_b['도실'] - wbc_lim_b['병살']
wbc_lim_b['B'] = wbc_lim_b['루타'] + 0.26*(wbc_lim_b['볼넷'] - wbc_lim_b['고4'] + wbc_lim_b['사구']) + 0.52*(wbc_lim_b['희타'] + wbc_lim_b['희비'] + wbc_lim_b['도루'])
wbc_lim_b['C'] = wbc_lim_b['타석'] + wbc_lim_b['볼넷'] + wbc_lim_b['사구'] + wbc_lim_b['희타'] + wbc_lim_b['희비']
wbc_lim_b['RC'] = wbc_lim_b['A']*wbc_lim_b['B']/wbc_lim_b['C']
wbc_lim_b['RC27'] = (wbc_lim_b['RC']/(wbc_lim_b['타석'] - wbc_lim_b['안타'] + wbc_lim_b['희타'] + wbc_lim_b['희비'] + wbc_lim_b['도실'] + wbc_lim_b['병살']))*27




# 데이터 변형

In [5]:
wbc_won_b['팀'] = '국가대표'
wbc_lim_b['팀'] = '국가대표'

In [6]:
wbc_won_p['팀'] = '국가대표'
wbc_lim_p['팀'] = '국가대표'

In [7]:
#필요한 지표들만 남김
#OPS 보정한 GPA로 대체
wbc_b2 = wbc_b[['팀', 'wOBA', 'wRC+', 'BWAR', 'BWPA', 'GPA', 'XR27','RC27']]
kbo_b2 = kbo_b[['팀','연도', 'wOBA', 'wRC+', 'BWAR', 'BWPA', 'GPA', 'XR27','RC27']]
wbc_won_b2 = wbc_won_b[['팀', 'wOBA', 'wRC+', 'BWAR', 'BWPA', 'GPA', 'XR27','RC27']]
wbc_lim_b2 = wbc_lim_b[['팀', 'wOBA', 'wRC+', 'BWAR', 'BWPA', 'GPA', 'XR27','RC27']]

In [8]:
print(wbc_p['이닝'].sum()) #1844 
print(wbc_b['타석'].sum()) #6970
print(kbo_p['이닝'].mean()) #1277
print(kbo_b['타석'].mean()) #5655 
print(wbc_won_p['이닝'].sum()) #1662
print(wbc_won_b['타석'].sum()) #6958
print(wbc_lim_p['이닝'].sum()) #1436
print(wbc_lim_b['타석'].sum()) #5020
#WAR, WPA는 합지표라 비율 맞춰줘야함

1843.6999999999998
6970
1277.4524999999999
5655.0875
1661.6000000000001
6958
1606.8
7225


In [9]:
wbc_p2 = wbc_p[['팀','ERA', 'FIP', 'WHIP', 'PWAR', 'PWPA']]
wbc_won_p2 = wbc_won_p[['팀','ERA', 'FIP', 'WHIP', 'PWAR', 'PWPA']]
wbc_lim_p2 = wbc_lim_p[['팀','ERA', 'FIP', 'WHIP', 'PWAR', 'PWPA']]
kbo_p2 = kbo_p[['팀', '연도', 'ERA', 'FIP', 'WHIP', 'PWAR', 'PWPA']]

In [10]:
# 단순 비율
wbc_b3 = wbc_b2
wbc_won_b3 = wbc_won_b2
wbc_lim_b3 = wbc_lim_b2

wbc_won_b3['BWAR'] = wbc_won_b2['BWAR']*5655/6958
wbc_won_b3['BWPA'] = wbc_won_b2['BWPA']*5655/6958
wbc_won_p2['PWAR'] = wbc_won_p2['PWAR']*1277/1662
wbc_won_p2['PWPA'] = wbc_won_p2['PWPA']*1277/1662

wbc_lim_b3['BWAR'] = wbc_lim_b2['BWAR']*5655/5020
wbc_lim_b3['BWPA'] = wbc_lim_b2['BWPA']*5655/5020
wbc_lim_p2['PWAR'] = wbc_lim_p2['PWAR']*1277/1436
wbc_lim_p2['PWPA'] = wbc_lim_p2['PWPA']*1277/1436

wbc_b3['BWAR'] = wbc_b2['BWAR']*5655/6970
wbc_b3['BWPA'] = wbc_b2['BWPA']*5655/6970
wbc_p2['PWAR'] = wbc_p2['PWAR']*1277/1844
wbc_p2['PWPA'] = wbc_p2['PWPA']*1277/1844

C:\Users\User\AppData\Local\Temp\ipykernel_1060\2048960649.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wbc_won_b3['BWAR'] = wbc_won_b2['BWAR']*5655/6958
C:\Users\User\AppData\Local\Temp\ipykernel_1060\2048960649.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wbc_won_b3['BWPA'] = wbc_won_b2['BWPA']*5655/6958
C:\Users\User\AppData\Local\Temp\ipykernel_1060\2048960649.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [11]:
from collections import defaultdict

kbo = ['LG', "두산", "키움", "SSG", "한화", "롯데", "삼성", "KT", "KIA", "NC"]

kbo_dict_b = defaultdict()
kbo_dict_p = defaultdict()


for i in range(len(kbo)):
    kbo_dict_b[kbo[i]] = kbo_b2[kbo_b2["팀"] == kbo[i]].sort_values(by = "연도")
    kbo_dict_p[kbo[i]] = kbo_p2[kbo_p2["팀"] == kbo[i]].sort_values(by = "연도")
    
LG_p = kbo_dict_p['LG']
DS_p = kbo_dict_p['두산']
KW_p = kbo_dict_p['키움']
SSG_p = kbo_dict_p['SSG']
HH_p = kbo_dict_p['한화']
LT_p = kbo_dict_p['롯데']
SS_p = kbo_dict_p['삼성']
KT_p = kbo_dict_p['KT']
KIA_p = kbo_dict_p['KIA']
NC_p = kbo_dict_p['NC']

LG_b = kbo_dict_b['LG']
DS_b = kbo_dict_b['두산']
KW_b = kbo_dict_b['키움']
SSG_b = kbo_dict_b['SSG']
HH_b = kbo_dict_b['한화']
LT_b = kbo_dict_b['롯데']
SS_b = kbo_dict_b['삼성']
KT_b = kbo_dict_b['KT']
KIA_b = kbo_dict_b['KIA']
NC_b = kbo_dict_b['NC']

LG = pd.merge(LG_b,LG_p)
DS = pd.merge(DS_b,DS_p)
KW = pd.merge(KW_b,KW_p)
SSG = pd.merge(SSG_b,SSG_p)
HH = pd.merge(HH_b,HH_p)
LT = pd.merge(LT_b,LT_p)
SS = pd.merge(SS_b,SS_p)
KT = pd.merge(KT_b,KT_p)
KIA = pd.merge(KIA_b,KIA_p)
NC = pd.merge(NC_b,NC_p)

LG["승 수"] = [64,71,69,68,79,79,72,87]
DS["승 수"] = [79,93,84,93,88,79,71,60]
KW["승 수"] = [78,77,69,75,86,80,70,80]
SSG["승 수"] = [69,69,75,78,88,51,66,88]
HH["승 수"] = [68,66,61,77,58,46,49,46]
LT["승 수"] = [66,66,80,68,48,71,65,64]
SS["승 수"] = [88,65,55,68,60,64,76,66]
KT["승 수"] = [52,53,50,59,71,81,76,80]
KIA["승 수"] = [67,70,87,70,62,73,58,70]
NC["승 수"] = [84,83,79,58,73,83,67,67] 

LG["승률"] = LG['승 수']/144
DS["승률"] = DS['승 수']/144
KW["승률"] = KW['승 수']/144
SSG["승률"] = SSG['승 수']/144
HH["승률"] = HH['승 수']/144
LT["승률"] = LT['승 수']/144
SS["승률"] = SS['승 수']/144
KT["승률"] = KT['승 수']/144
KIA["승률"] = KIA['승 수']/144
NC["승률"] = NC['승 수']/144

In [12]:
#데이터 세로로 다시 병합
kbo_team = [LG, DS, KW, SSG, HH, LT, SS, KT, KIA, NC]
kbo_re = pd.concat(kbo_team)

In [13]:
wbc = []

wbc.append(wbc_b3['wOBA'].mean())
wbc.append(wbc_b3['wRC+'].mean())
wbc.append(wbc_b3['BWAR'].sum())
wbc.append(wbc_b3['BWPA'].sum())
wbc.append(wbc_b3['GPA'].mean())
wbc.append(wbc_b3['XR27'].mean())
wbc.append(wbc_b3['RC27'].mean())
wbc.append(wbc_p2['ERA'].mean())
wbc.append(wbc_p2['FIP'].mean())
wbc.append(wbc_p2['WHIP'].mean())
wbc.append(wbc_p2['PWAR'].sum())
wbc.append(wbc_p2['PWPA'].sum())
print(wbc)

wbc_won = []

wbc_won.append(wbc_won_b3['wOBA'].mean())
wbc_won.append(wbc_won_b3['wRC+'].mean())
wbc_won.append(wbc_won_b3['BWAR'].sum())
wbc_won.append(wbc_won_b3['BWPA'].sum())
wbc_won.append(wbc_won_b3['GPA'].mean())
wbc_won.append(wbc_won_b3['XR27'].mean())
wbc_won.append(wbc_won_b3['RC27'].mean())
wbc_won.append(wbc_won_p2['ERA'].mean())
wbc_won.append(wbc_won_p2['FIP'].mean())
wbc_won.append(wbc_won_p2['WHIP'].mean())
wbc_won.append(wbc_won_p2['PWAR'].sum())
wbc_won.append(wbc_won_p2['PWPA'].sum())
print(wbc_won)

wbc_lim = []

wbc_lim.append(wbc_lim_b3['wOBA'].mean())
wbc_lim.append(wbc_lim_b3['wRC+'].mean())
wbc_lim.append(wbc_lim_b3['BWAR'].sum())
wbc_lim.append(wbc_lim_b3['BWPA'].sum())
wbc_lim.append(wbc_lim_b3['GPA'].mean())
wbc_lim.append(wbc_lim_b3['XR27'].mean())
wbc_lim.append(wbc_lim_b3['RC27'].mean())
wbc_lim.append(wbc_lim_p2['ERA'].mean())
wbc_lim.append(wbc_lim_p2['FIP'].mean())
wbc_lim.append(wbc_lim_p2['WHIP'].mean())
wbc_lim.append(wbc_lim_p2['PWAR'].sum())
wbc_lim.append(wbc_lim_p2['PWPA'].sum())
print(wbc_lim)

[0.3723846153846154, 132.46153846153845, 47.68211621233859, 21.800552367288386, 0.27926923076923077, 4.843267817549409, 4.776920969422313, 3.0880000000000005, 3.3786666666666667, 1.2233333333333334, 29.61892082429501, 14.535916485900218]
[0.38323076923076915, 139.23846153846154, 49.92622161540673, 25.934327392929006, 0.28758076923076925, 5.1071871226875345, 4.94067774969287, 2.6146666666666665, 3.1799999999999997, 1.162, 41.61391095066185, 26.3160348977136]
[0.3782307692307692, 136.13846153846154, 69.92148406374501, 37.92905378486056, 0.2843846153846154, 4.992984902631808, 4.846307308173776, 2.7806666666666664, 3.0893333333333333, 1.1500000000000001, 42.258384401114206, 22.703210306406685]


# 데이터 스케일링

In [14]:
pd.options.mode.chained_assignment = None

def scaling(df, scale_columns):
    for i in scale_columns:
        series_mean = df[i].mean()
        series_std = df[i].std()
        df[i] = df[i].apply(lambda x : (x - series_mean)/series_std)
    return df

In [15]:
kbo_re2 = kbo_re[kbo_re.columns.difference(['팀', '연도', '승 수', '승률'])]

# wbc 스케일링 해주기 위해 병합

In [16]:
wbc_re = pd.DataFrame([wbc, wbc_won, wbc_lim], columns = ['wOBA', 'wRC+', 'BWAR', 'BWPA', 'GPA', 'XR27',
       'RC27', 'ERA', 'FIP', 'WHIP', 'PWAR', 'PWPA'], dtype = float)

In [17]:
kbo_scale = pd.concat([kbo_re2, wbc_re])

In [18]:
column = wbc_re.columns
scaling(kbo_scale, column)

,BWAR,BWPA,ERA,FIP,GPA,PWAR,PWPA,RC27,WHIP,XR27,wOBA,wRC+
0,-0.777360,-0.744637,-0.015521,-0.234610,-0.579863,0.688159,-0.125988,-0.566527,-0.159386,-0.632221,-0.835042,-0.823948
1,-0.258858,0.616450,0.647840,0.690332,0.352493,-0.531078,-0.748446,0.353546,0.670115,0.426651,0.422155,-0.306018
2,-0.720439,-0.416651,-0.476990,-0.409128,-0.301451,0.215898,0.045413,-0.449688,-1.265386,-0.638469,-0.663606,-0.744942
3,0.036098,0.696576,0.965099,0.672881,0.572632,-0.728625,-1.186471,0.797393,0.301448,0.651313,0.136428,0.089013
4,-0.098443,-0.525624,-1.097088,-0.880325,-1.094601,-0.452367,1.100885,-0.937185,-0.804553,-1.038374,-1.006478,-0.235790
...,...,...,...,...,...,...,...,...,...,...,...,...
6,0.107509,-0.316226,-0.159730,-0.269514,-0.188144,-0.169937,0.043409,-0.316980,0.209281,-0.422663,0.250719,0.264583
7,-0.192622,-1.584365,-1.068246,-1.072294,-1.114025,0.147991,0.957549,-1.092035,-0.896719,-1.139545,-1.063623,-0.086556
0,2.576046,1.527817,-2.253643,-2.243888,1.166311,1.830062,2.028649,1.581159,-2.156331,1.548733,1.587040,2.719178
1,2.808297,1.969451,-2.936232,-2.590596,1.704458,3.681293,3.209426,1.884729,-2.721621,2.053682,2.206847,3.314088


# kbo 따로 스케일링

In [19]:
scaling(kbo_re, kbo_re.columns.difference(['팀', '연도', '승 수', '승률']))

,팀,연도,wOBA,wRC+,BWAR,BWPA,GPA,XR27,RC27,ERA,FIP,WHIP,PWAR,PWPA,승 수,승률
0,LG,15,-0.808258,-0.864681,-0.886317,-0.729899,-0.538176,-0.592500,-0.523161,-0.130660,-0.371893,-0.290637,1.013789,-0.028921,64,0.444444
1,LG,16,0.522457,-0.234365,-0.178309,0.774688,0.417107,0.519385,0.436019,0.628945,0.672362,0.651970,-0.522992,-0.751279,71,0.493056
2,LG,17,-0.626797,-0.768531,-0.808592,-0.367333,-0.252918,-0.599061,-0.401356,-0.659081,-0.568922,-1.547448,0.418529,0.169990,69,0.479167
3,LG,18,0.220022,0.246384,0.224450,0.863262,0.642660,0.755295,0.898731,0.992234,0.652659,0.233034,-0.771989,-1.259605,68,0.472222
4,LG,19,-0.989719,-0.148899,0.040735,-0.487795,-1.065571,-1.018988,-0.909573,-1.369147,-1.100901,-1.023777,-0.423782,1.394858,79,0.548611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,NC,18,-1.292154,-1.772764,-1.677703,-0.080352,-0.753778,-0.711317,-0.543560,1.289471,1.618102,1.070907,-1.172719,-1.482943,58,0.402778
4,NC,19,0.159535,0.801917,0.689389,-0.925944,-0.077119,-0.146350,-0.122894,-1.154476,-0.982683,-0.919043,0.074213,1.144766,73,0.506944
5,NC,20,1.611223,1.571116,1.650358,1.308497,1.312684,1.333718,1.431762,-0.213226,0.357115,-0.500106,0.484669,0.386348,83,0.576389
6,NC,21,0.340996,0.460051,0.321960,-0.256320,-0.136824,-0.372451,-0.263007,-0.295792,-0.411298,0.128299,-0.067793,0.167663,67,0.465278


In [20]:
kbo_re.drop(columns = ['팀', '연도', '승 수'], axis = True, inplace = True)

# Auto ML

In [21]:
#pip install mljar-supervised


In [22]:
from supervised.automl import AutoML

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

In [24]:
x_train, x_val, y_train, y_val = train_test_split(
    kbo_re[kbo_re.columns.difference(['승률'])],
    kbo_re['승률'],
    test_size=0.2,
    random_state=123,
)

In [25]:
automl = AutoML(mode="Explain")
automl.fit(x_train, y_train)

AutoML directory: AutoML_1
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Baseline', 'Linear', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 3 models
1_Baseline rmse 0.05892 trained in 0.26 seconds
2_DecisionTree rmse 0.05194 trained in 10.32 seconds
3_Linear rmse 0.00998 trained in 5.55 seconds
* Step default_algorithms will try to check up to 3 models
4_Default_Xgboost rmse 0.042384 trained in 1.37 seconds
5_Default_NeuralNetwork rmse 0.142022 trained in 0.29 seconds
6_Default_RandomForest rmse 0.052102 trained in 1.56 seconds
* Step ensemble will try to check up to 1 model
Ensemble rmse 0.00998 trained in 0.15 seconds
AutoML fit time: 23.57 seconds
AutoML best model: 3_Linear


AutoML()

In [26]:
y_pred = automl.predict(x_val)
print(mean_absolute_error(y_val, y_pred))
print(r2_score(y_val, y_pred))

0.008533355145155713
0.9891201363029438


In [27]:
y_pred

array([0.46475576, 0.33929119, 0.42880803, 0.5393905 , 0.49506463,
       0.44986168, 0.60283069, 0.58329016, 0.53662912, 0.48418779,
       0.63950576, 0.30901816, 0.40409567, 0.33589579, 0.58610179,
       0.46104093])

In [28]:
x_test = kbo_scale.iloc[80:]

In [29]:
automl.predict(x_test)

array([0.70363501, 0.80236036, 0.85119916])